In [1]:
!pip install -q google-generativeai sentence-transformers scikit-learn lexicalrichness transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import google.generativeai as genai
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from lexicalrichness import LexicalRichness
import numpy as np
import json
from collections import Counter
import time

In [ ]:
GEMINI_API_KEY = ""
genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel('gemini-2.5-flash')

In [4]:
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer.pad_token = tokenizer.eos_token
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
category_weights = {
    'factual': [0.15, 0.2, 0.15, 0.4, 0.1],
    'mathematical': [0.1, 0.5, 0.2, 0.15, 0.05],
    'reasoning': [0.15, 0.15, 0.2, 0.4, 0.1]
}

calibration_stats = {
    'semantic_entropy': {'min': 0.0, 'max': 1.6},
    'self_consistency': {'min': 0.0, 'max': 1.0},
    'token_entropy': {'min': 0.0, 'max': 5.0},
    'perplexity': {'min': 1.0, 'max': 150.0},
    'lexical_diversity': {'min': 0.0, 'max': 1.0}
}

In [6]:
def semantic_entropy(responses):
    if len(responses) < 2:
        return 0.0

    embeddings = embedding_model.encode(responses)
    clustering = DBSCAN(eps=0.5, min_samples=1, metric='cosine').fit(embeddings)
    labels = clustering.labels_

    cluster_counts = Counter(labels)
    total = len(labels)
    probs = [count / total for count in cluster_counts.values()]

    entropy = -sum(p * np.log(p + 1e-12) for p in probs if p > 0)
    return float(entropy)

def self_consistency(responses):
    if len(responses) < 2:
        return 0.0

    normalized = [r.strip().lower()[:50] for r in responses]
    most_common = Counter(normalized).most_common(1)[0][1]
    consistency = most_common / len(responses)

    return 1 - consistency

def token_entropy_approx(responses):
    all_tokens = []
    for response in responses:
        tokens = response.lower().split()
        all_tokens.extend(tokens)

    if len(all_tokens) == 0:
        return 0.0

    token_counts = Counter(all_tokens)
    total = len(all_tokens)
    probs = [count / total for count in token_counts.values()]

    entropy = -sum(p * np.log(p + 1e-12) for p in probs if p > 0)
    return float(entropy)

def perplexity_approx(responses):
    token_ent = token_entropy_approx(responses)
    return float(np.exp(token_ent))

def lexical_diversity(responses):
    combined_text = " ".join(responses)

    if len(combined_text.strip()) == 0:
        return 0.0

    lex = LexicalRichness(combined_text)
    try:
        ttr = lex.ttr
    except:
        ttr = 0.0

    return float(ttr)

In [7]:
def normalize_metric(value, metric_name):
    stats = calibration_stats[metric_name]
    normalized = (value - stats['min']) / (stats['max'] - stats['min'] + 1e-8)
    return np.clip(normalized, 0.0, 1.0)

def calculate_all_metrics(responses):
    raw_metrics = {
        'semantic_entropy': semantic_entropy(responses),
        'self_consistency': self_consistency(responses),
        'token_entropy': token_entropy_approx(responses),
        'perplexity': perplexity_approx(responses),
        'lexical_diversity': lexical_diversity(responses)
    }

    normalized_metrics = {
        name: normalize_metric(value, name)
        for name, value in raw_metrics.items()
    }

    return raw_metrics, normalized_metrics

In [8]:
def categorize_query(query):
    prompt = f"""Analyze this query and determine what percentage it falls into each category.
Provide scores that sum to 1.0.

Query: "{query}"

Categories:
- factual: Objective, verifiable facts
- mathematical: Numerical computations and math problems
- reasoning: Logical deduction, multi-hop reasoning, common sense

Respond ONLY with valid JSON in this exact format (no markdown, no backticks):
{{"factual": 0.0, "mathematical": 0.0, "reasoning": 0.0}}
"""

    try:
        response = gemini_model.generate_content(prompt)
        response_text = response.text.strip()

        response_text = response_text.replace('```json', '').replace('```', '').strip()

        category_scores = json.loads(response_text)

        total = sum(category_scores.values())
        if abs(total - 1.0) > 0.1:
            for key in category_scores:
                category_scores[key] /= total

        return category_scores

    except Exception as e:
        print(f"Categorization failed: {e}, defaulting to reasoning")
        return {'factual': 0.0, 'mathematical': 0.0, 'reasoning': 1.0}

In [9]:
def blend_weights(category_scores):
    final_weights = np.zeros(5)

    for category, score in category_scores.items():
        if score > 0 and category in category_weights:
            final_weights += score * np.array(category_weights[category])

    if final_weights.sum() > 0:
        final_weights = final_weights / final_weights.sum()
    else:
        final_weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

    return final_weights

In [10]:
def generate_responses(query, num_samples=5):
    responses = []

    prompt = f"Answer this question concisely: {query}\nAnswer:"

    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.split("Answer:")[-1].strip()
        responses.append(response)

    return responses

In [11]:
def calculate_uncertainty(query, verbose=True):
    if verbose:
        print(f"Query: {query}\n")

    category_scores = categorize_query(query)
    if verbose:
        print("Category Distribution:")
        for cat, score in category_scores.items():
            print(f"  {cat}: {score:.2%}")
        print()

    blended_weights = blend_weights(category_scores)
    if verbose:
        metric_names = ['Semantic Entropy', 'Self-Consistency', 'Token Entropy', 'Perplexity', 'Lexical Diversity']
        print("Blended Weights:")
        for name, weight in zip(metric_names, blended_weights):
            print(f"  {name}: {weight:.3f}")
        print()

    responses = generate_responses(query)
    if verbose:
        print(f"Generated {len(responses)} responses")
        print(f"Sample response: {responses[0][:100]}...\n")

    raw_metrics, normalized_metrics = calculate_all_metrics(responses)

    if verbose:
        print("Normalized Metrics:")
        for name, value in normalized_metrics.items():
            print(f"  {name}: {value:.3f}")
        print()

    metric_values = [
        normalized_metrics['semantic_entropy'],
        normalized_metrics['self_consistency'],
        normalized_metrics['token_entropy'],
        normalized_metrics['perplexity'],
        normalized_metrics['lexical_diversity']
    ]

    final_uncertainty = float(np.dot(blended_weights, metric_values))

    if verbose:
        print(f"Final Uncertainty Score: {final_uncertainty:.3f}")

        if final_uncertainty < 0.3:
            print("Interpretation: 🟢 Low Uncertainty - Trust this answer")
        elif final_uncertainty < 0.7:
            print("Interpretation: 🟡 Medium Uncertainty - Verify if important")
        else:
            print("Interpretation: 🔴 High Uncertainty - Don't trust this answer")

    return {
        'query': query,
        'category_scores': category_scores,
        'responses': responses,
        'raw_metrics': raw_metrics,
        'normalized_metrics': normalized_metrics,
        'blended_weights': blended_weights.tolist(),
        'uncertainty_score': final_uncertainty
    }

In [12]:
result1 = calculate_uncertainty("What is the capital of France?")

Query: What is the capital of France?

Category Distribution:
  factual: 100.00%
  mathematical: 0.00%
  reasoning: 0.00%

Blended Weights:
  Semantic Entropy: 0.150
  Self-Consistency: 0.200
  Token Entropy: 0.150
  Perplexity: 0.400
  Lexical Diversity: 0.100

Generated 5 responses
Sample response: The capital of France is Paris....

Normalized Metrics:
  semantic_entropy: 0.000
  self_consistency: 0.200
  token_entropy: 0.392
  perplexity: 0.041
  lexical_diversity: 0.200

Final Uncertainty Score: 0.135
Interpretation: 🟢 Low Uncertainty - Trust this answer


In [13]:
result2 = calculate_uncertainty("If John has 5 apples and buys 3 more, how many does he have?")

Query: If John has 5 apples and buys 3 more, how many does he have?

Category Distribution:
  factual: 0.00%
  mathematical: 95.00%
  reasoning: 5.00%

Blended Weights:
  Semantic Entropy: 0.103
  Self-Consistency: 0.483
  Token Entropy: 0.200
  Perplexity: 0.163
  Lexical Diversity: 0.053

Generated 5 responses
Sample response: 8 apples....

Normalized Metrics:
  semantic_entropy: 0.000
  self_consistency: 0.400
  token_entropy: 0.271
  perplexity: 0.019
  lexical_diversity: 0.273

Final Uncertainty Score: 0.265
Interpretation: 🟢 Low Uncertainty - Trust this answer


In [14]:
result3 = calculate_uncertainty("What government position was held by the woman who portrayed Corliss Archer?")

Query: What government position was held by the woman who portrayed Corliss Archer?

Category Distribution:
  factual: 40.00%
  mathematical: 0.00%
  reasoning: 60.00%

Blended Weights:
  Semantic Entropy: 0.150
  Self-Consistency: 0.170
  Token Entropy: 0.180
  Perplexity: 0.400
  Lexical Diversity: 0.100

Generated 5 responses
Sample response: The actress who played Corliss Archer, Lauren Hutton, held the government position of Governor of Ca...

Normalized Metrics:
  semantic_entropy: 0.594
  self_consistency: 0.800
  token_entropy: 0.915
  perplexity: 0.643
  lexical_diversity: 0.471

Final Uncertainty Score: 0.694
Interpretation: 🟡 Medium Uncertainty - Verify if important


In [15]:
def quick_uncertainty(query):
    result = calculate_uncertainty(query, verbose=False)
    print(f"Query: {result['query']}")
    print(f"Response: {result['responses'][0][:150]}")
    print(f"Uncertainty: {result['uncertainty_score']:.3f}")

    if result['uncertainty_score'] < 0.3:
        print("Status: 🟢 Confident")
    elif result['uncertainty_score'] < 0.7:
        print("Status: 🟡 Uncertain")
    else:
        print("Status: 🔴 Very Uncertain")
    print()

quick_uncertainty("What is 2+2?")
quick_uncertainty("Who won the 2023 FIFA World Cup?")
quick_uncertainty("What will the stock market do tomorrow?")

Query: What is 2+2?
Response: 2 + 2 equals 4. Answer. 4

To break it down further:

1. The question asks for the sum of 2 and 2.
2. When you add 2 and 2 together, you are essential
Uncertainty: 0.683
Status: 🟡 Uncertain

Query: Who won the 2023 FIFA World Cup?
Response: The 2023 FIFA World Cup has not been held yet, as it was scheduled for November to December 2026. Therefore, there is no winner as of now. Answer this
Uncertainty: 0.587
Status: 🟡 Uncertain

Query: What will the stock market do tomorrow?
Response: As an AI, I don't have real-time data or predictive capabilities to forecast stock market movements accurately. Stock market trends are influenced by 
Uncertainty: 0.684
Status: 🟡 Uncertain



In [16]:
result4 = calculate_uncertainty('''Proov the following: The following assertions are equivalent for graphs G:
(i) G is planar;
(ii) G contains neither K5 nor K3,3 as a minor;
(iii) G contains neither K5 nor K3,3 as a topological minor.''')

Query: Proov the following: The following assertions are equivalent for graphs G:
(i) G is planar;
(ii) G contains neither K5 nor K3,3 as a minor;
(iii) G contains neither K5 nor K3,3 as a topological minor.

Category Distribution:
  factual: 10.00%
  mathematical: 30.00%
  reasoning: 60.00%

Blended Weights:
  Semantic Entropy: 0.135
  Self-Consistency: 0.260
  Token Entropy: 0.195
  Perplexity: 0.325
  Lexical Diversity: 0.085

Generated 5 responses
Sample response: The given assertions are equivalent for graphs G in the context of graph theory:

(i) G is planar
(i...

Normalized Metrics:
  semantic_entropy: 0.000
  self_consistency: 0.800
  token_entropy: 0.884
  perplexity: 0.551
  lexical_diversity: 0.305

Final Uncertainty Score: 0.585
Interpretation: 🟡 Medium Uncertainty - Verify if important


In [17]:
result5 = calculate_uncertainty("What does 'bank' mean?")

Query: What does 'bank' mean?

Category Distribution:
  factual: 100.00%
  mathematical: 0.00%
  reasoning: 0.00%

Blended Weights:
  Semantic Entropy: 0.150
  Self-Consistency: 0.200
  Token Entropy: 0.150
  Perplexity: 0.400
  Lexical Diversity: 0.100

Generated 5 responses
Sample response: In Douglas Adams' series "The Hitchhiker's Guide to the Galaxy," the answer to life, the universe, a...

Normalized Metrics:
  semantic_entropy: 0.313
  self_consistency: 0.800
  token_entropy: 0.810
  perplexity: 0.378
  lexical_diversity: 0.557

Final Uncertainty Score: 0.535
Interpretation: 🟡 Medium Uncertainty - Verify if important


In [18]:
!pip install -q gradio

In [19]:
import gradio as gr

In [20]:
def process_query(query):
    if not query.strip():
        return "Please enter a query.", ""

    try:
        result = calculate_uncertainty(query, verbose=False)

        response_text = f"**Answer:** {result['responses'][0]}\n\n"

        uncertainty = result['uncertainty_score']

        if uncertainty < 0.3:
            status = "🟢 **Low Uncertainty** - Trust this answer"
        elif uncertainty < 0.7:
            status = "🟡 **Medium Uncertainty** - Verify if important"
        else:
            status = "🔴 **High Uncertainty** - Don't trust this answer"

        output = f"{response_text}**Uncertainty Score:** {uncertainty:.3f}\n\n{status}"

        return output

    except Exception as e:
        return f"Error: {str(e)}"

In [21]:
demo = gr.Interface(
    fn=process_query,
    inputs=gr.Textbox(label="Enter your query", placeholder="Type any question...", lines=2),
    outputs=gr.Markdown(label="Result"),
    title="🎯 LLM Uncertainty Quantification",
    description="Ask any question and get the model's response with an uncertainty score.",
    examples=[
        ["What is the capital of France?"],
        ["What is 25% of 80?"],
        ["What color shirt was I wearing yesterday?"]
    ],
    allow_flagging="never"
)

demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a5d497c161cf01da46.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
